# Run workflow (pipeline)

## CSV to pickle
```bash
python -m src.preprocessing.csv2pkl \
    --input_dir data/raw/ \
    --output_dir data/processed_pickle_2/ \
    --cargo_tankers_only \
    --run_name alex_local_csv2pkl
```

---

## Map_reduce
### V2
```bash
python -m src.preprocessing.map_reduce \
    --input_dir data/processed_pickle_2/ \
    --output_dir data/processed_final/map_reduced/ \
    --num_workers 0 \
    --run_name alex_local_map_reduce
```

---

## Train test split
```bash
python -m src.preprocessing.train_test_split \
    --data_dir data/processed_final/map_reduced/ \
    --val_size 0.1 \
    --test_size 0.1 \
    --random_state 42
```


---

## Train Models
### V3
```bash
# TPtrans
python -m src.train.train_tptrans_transformer --config configs/traj_tptrans.yaml
# Traisformer
python -m src.train.train_traisformer --config configs/traj_traisformer.yaml
# Kalmar ????
```

---


## Evaluate model

Run src.eval.evaluate_trajectory and look at produced map_model.html file to see interactive map.

#### Example 
```bash
python -m src.eval.evaluate_trajectory \
  --split_dir data/processed_final/map_reduced/test \
  --ckpt data/checkpoints/tptrans_delta_v3/tptrans_medium/traj_tptrans_delta.pt,data/checkpoints/traisformer_new_small/traj_traisformer.pt \
  --model tptrans,traisformer,kalman \
  --out_dir data/figures/final/all_models_test \
  --pred_cut 80 --folium --same_pic --collect \
  --samples 1 --temperature 0 --top_k 20 --mmsi 212801000,215933000,218615000,230617000,244554000,248891000,250005981,255802840,305575000,305643000,352005235,636015943,636022355
```

#### TPtrans
```bash
python -m src.eval.evaluate_trajectory \
  --split_dir data/processed_final/map_reduced/test \
  --ckpt data/checkpoints/tptrans_delta_v3/tptrans_medium/traj_tptrans_delta.pt \
  --model tptrans \
  --out_dir data/figures/final/eval_tptrans_medium_v3_80 \
  --pred_cut 80 \
  --folium \
  --same_pic \
  --collect
```

#### Traisformer
```bash
python -m src.eval.evaluate_trajectory \
  --split_dir data/processed_final/map_reduced/test \
  --ckpt data/checkpoints/traisformer_new_small/traj_traisformer.pt \
  --model traisformer \
  --out_dir data/figures/final/eval_traisformer_80_small_temp0_topk20 \
  --pred_cut 80 --same_pic --folium \
  --samples 1 --temperature 0 --top_k 20 \
  --prevent_stuck --collect
```

#### kalman filter
```bash
python -m src.eval.evaluate_trajectory \
  --split_dir data/processed_final/map_reduced/test  \
  --model kalman \
  --out_dir data/figures/final/eval_kalman_80 \
  --pred_cut 80 \
  --same_pic --folium --collect 
```

#### All Models 
```bash
python -m src.eval.evaluate_trajectory \
  --split_dir data/processed_final/map_reduced/test \
  --ckpt data/checkpoints/tptrans_delta_v3/tptrans_medium/traj_tptrans_delta.pt,data/checkpoints/traisformer_new_small/traj_traisformer.pt \
  --model tptrans,traisformer,kalman \
  --out_dir data/figures/final/all_models_test \
  --pred_cut 80 --folium --same_pic --collect \
  --samples 1 --temperature 0 --top_k 20 
```

#### Usage:
##### MMSI subset:
```bash
  --mmsi 212801000,215933000,218615000,230617000,244554000,248891000,250005981,255802840,305575000,305643000,352005235,636015943,636022355
```
##### All Args
```bash
evaluate_trajectory.py [-h] --split_dir SPLIT_DIR --out_dir OUT_DIR --model MODEL
                              [--ckpt CKPT] [--lat_min LAT_MIN] [--lat_max LAT_MAX]
                              [--lon_min LON_MIN] [--lon_max LON_MAX] [--speed_max SPEED_MAX]
                              [--plot_lat_min PLOT_LAT_MIN] [--plot_lat_max PLOT_LAT_MAX]
                              [--plot_lon_min PLOT_LON_MIN] [--plot_lon_max PLOT_LON_MAX]
                              [--past_len PAST_LEN] [--pred_cut PRED_CUT] [--pred_len PRED_LEN]
                              [--cap_future CAP_FUTURE] [--horizon HORIZON]
                              [--pred_scale PRED_SCALE] [--samples SAMPLES]
                              [--temperature TEMPERATURE] [--top_k TOP_K] [--prevent_stuck]
                              [--no_plots] [--same_pic] [--folium] [--collect] [--mmsi MMSI]
                              [--cpu] [--dpi DPI]
```

HPC Usage:

Remember to set DTU blackhole path
```bash
  --split_dir /dtu/blackhole/10/178320/preprocessed_2/final/test \
```

In [7]:
kalman = "/Users/alexanderschiotz/Desktop/DTU/Master/Deep Learning/projects/ais-mda/data/figures/final/eval_kalman_80/eval_collection.pkl"
tptrans = "/Users/alexanderschiotz/Desktop/DTU/Master/Deep Learning/projects/ais-mda/data/figures/final/eval_tptrans_medium_v3_80/eval_collection.pkl"
traisformer = "/Users/alexanderschiotz/Desktop/DTU/Master/Deep Learning/projects/ais-mda/data/figures/final/eval_traisformer_80_small_temp0_topk20/eval_collection.pkl"
all = "/Users/alexanderschiotz/Desktop/DTU/Master/Deep Learning/projects/ais-mda/data/figures/final/all_models_test/predictions_tptrans_traisformer_kalman.pkl"

In [ ]:
import pandas as pd

# Read the pickle files
df_kalman = pd.read_pickle(kalman)
df_tptrans = pd.read_pickle(tptrans)
df_traisformer = pd.read_pickle(traisformer)
df_all = pd.read_pickle(all)

# Display predictions
print("Kalman Predictions:")
print(df_kalman)

print("\nTPTrans Predictions:")
print(df_tptrans)

print("\nTraisformer Predictions:")
print(df_traisformer)

print("\nAll Models Predictions:")
print(df_all)